In [1]:
# Big TODOs

# Add typing to function and scripts

In [2]:
import os
import json
import re
import pandas as pd

# Extracting Recipes from Markdown Files

In [3]:
def extract_recipes(recipes, exclude):
    """
    Extract all the recipes from the local directory
    
    recipes: 
    """
    ing_dir = {key: {} for key in recipes}

    for filename in recipes:
        if filename.startswith("."):
            continue
        elif filename.endswith("md"):
            with open (filename, "r") as f:
                recipe = f.read()
                ingredients, directions = recipe.split("## Ingredients")[1].split("## Directions")
                ing_dir[filename]["ingredients"] = ingredients
                ing_dir[filename]["directions"] = directions

In [4]:
exclude = ["francesinha.md"]
recipe_docs = [x for x in os.listdir(".") if x not in exclude]

if os.path.exists("./recipes.json"):
    f = open("recipes.json", "r")
    recipes = json.loads(f.read())
    f.close()
else:
    recipes = extract_recipes(recipe_docs, exclude)

# Cleaning Up Data

In [5]:
# TODO: Should only replace exact matches but it just replaces first match.
# Currently resolved by putting plural before singular in ls argument

def recursive_elimination(string, ls):
    """
    Recursively remove unwanted words from a string
    
    string: target from which you want to eliminate all instances of elements in ls
    ls: list of values to remove from string
    """
    for elem in ls:
        match = re.findall(elem, string)
        
        if len(match) != 0:
            for m in match:
                string = string.replace(m, "").strip()
            
    return string

In [6]:
# Not sure why there was empty ingredients in first place
ingredients = [list(x.values())[0].lower().strip().split("\n") for x in list(recipes.values()) if len(list(x.keys())) != 0]
ingredients = [val.strip("* ") for sublist in ingredients for val in sublist if val.startswith("*")]

# TODO: Move this to external JSON later
measurements = ["teaspoons", "teaspoon", "tablespoons", "tablespoon", "lb", "kg", "oz", "ml"]
size = ["small", "medium", "large"]
elim = measurements + size

for idx, x in enumerate(ingredients):
    match = re.findall(r"~*\d+\S*\d*\b", x)

    if len(match) != 0:
        for x in match:
            ingredients[idx] = ingredients[idx].replace(x, "").strip()

    # ingredients[idx] = recursive_elimination(x, elim)

In [7]:
# df = pd.DataFrame(ingredients)
# pd.DataFrame.to_csv(df, "./partially_cleaned_ingredients.csv", index=False)
# pd.DataFrame.to_csv(df, "./cleaned_ingredients.csv", index=False)

Data was cleaned by hand via Excel. Obviously this is not a sustainable way to do this, but it works for now.

In [8]:
manually_cleaned_ingredients = pd.read_csv("./manually_cleaned_ingredients.csv")

# Incredibly dumb but it works
ingredients_set = list(set(list(manually_cleaned_ingredients.iloc[:,0])))
len(ingredients_set)

117

,filename,ingredients,test
0,creamy-mashed-potatoes.md,The quantities here are for about four adult p...,0
1,red-sauce.md,- 1/3 lb salt pork \n- 2 lb chuck roast\n- 3 C...,0
2,tuscan-style-pork-roast.md,- 1 pork Roast\n- 2-3 Tbsp fresh rosemary\n- 8...,0
3,spicy-mayo.md,- chili sauce (e.g Sriracha)\n- mayonnaise\n- ...,0
4,pilaf.md,- 150ml cooking oil\n- 1kg long grain [rice](r...,0
...,...,...,...
244,banana-pancakes.md,- 1 cup flour (2.5 dl)\n- 1 tablespoon sugar\n...,0
245,pizza-dough.md,- 400g flour (basic wheat flour)\n- 200ml luke...,0
246,sourdough-potato-bread.md,- 150g boiled potatoes in there skin\n- 350g w...,0
247,sausage-rolls.md,- Eggs\n- Puff pastry\n- Pork meat / sausage meat,0


In [35]:
def recipe_encoding(recipes, ingredients_set):
    """
    One hot encoding of recipes based on ingredients they contain
    
    recipes: dictionary of recipes with an ingredients subdictionary
    ingredients_set: set of all ingredients
    """
    recipe_ingredients_tuples = [
        (name, val["ingredients"].strip()) 
        for name, val in recipes.items() 
        if "ingredients" in list(val.keys())
    ]
    recipe_df = pd.DataFrame(recipe_ingredients_tuples, columns=["filename", "ingredients"])
    
    for ingredient in ingredients_set:
        recipe_df[ingredient] = recipe_df["ingredients"].str.contains(ingredient)
        recipe_df[ingredient] = recipe_df[ingredient].astype("uint8")
        
    return recipe_df

In [37]:
recipe_df = recipe_encoding(recipes, ingredients_set)
recipe_df.head()

,filename,ingredients,whole kernel corn,eggs,bell pepper,unbleached all-purpose flour,ground beef,"tomatillos, husks removed",carrot,cup sugar,...,water,buckwheat,frozen peas,walnuts,butter,fed,oregano,shortening,pinto beans,brown sugar
0,creamy-mashed-potatoes.md,The quantities here are for about four adult p...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,red-sauce.md,- 1/3 lb salt pork \n- 2 lb chuck roast\n- 3 C...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,tuscan-style-pork-roast.md,- 1 pork Roast\n- 2-3 Tbsp fresh rosemary\n- 8...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,spicy-mayo.md,- chili sauce (e.g Sriracha)\n- mayonnaise\n- ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,pilaf.md,- 150ml cooking oil\n- 1kg long grain [rice](r...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
pd.DataFrame.to_csv(recipe_df, "encoded_recipes.csv", index=False)